# Technical Factor - BIAS

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_avg(close, 3) - m_avg(close, 6) AS BIAS,
        (m_avg(close, 3) - m_avg(close, 6)) / m_avg(close, 6) AS BIAS_RATE,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(BIAS > 0 AND m_lag(BIAS, 1) <= 0, 1, 0) AS TRBY1,
        IF(BIAS < 0 AND m_lag(BIAS, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(BIAS > 0 AND (BIAS - m_lag(BIAS, 1)) > 0, 1, 0) AS MTBY1,
        IF(BIAS < 0 AND (BIAS - m_lag(BIAS, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(BIAS_RATE < -0.05 AND (BIAS_RATE - m_lag(BIAS_RATE, 1)) > 0, 1, 0) AS RVBY1,
        IF(BIAS_RATE > 0.05 AND (BIAS_RATE - m_lag(BIAS_RATE, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(BIAS, 1) <= 0 AND BIAS > 0 AND (BIAS - m_lag(BIAS, 1)) > 0, 1, 0) AS BKBY1,
        IF(m_lag(BIAS, 1) >= 0 AND BIAS < 0 AND (BIAS - m_lag(BIAS, 1)) < 0, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        BIAS,
        BIAS_RATE,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,BIAS,BIAS_RATE,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-09,600519.SH,-103.588829,-0.008740,0,0,0,0,0,0,0,0
1,2025-01-10,600519.SH,-44.395213,-0.003768,0,0,0,0,0,0,0,0
2,2025-01-13,600519.SH,1.737794,0.000148,1,0,1,0,0,0,1,0
3,2025-01-14,600519.SH,35.000262,0.002970,0,0,1,0,0,0,0,0
4,2025-01-15,600519.SH,88.586777,0.007491,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
233,2025-12-25,600519.SH,-3.336562,-0.000281,0,0,0,0,0,0,0,0
234,2025-12-26,600519.SH,4.336123,0.000364,1,0,1,0,0,0,1,0
235,2025-12-29,600519.SH,18.696009,0.001572,0,0,1,0,0,0,0,0
236,2025-12-30,600519.SH,-24.045770,-0.002026,0,1,0,1,0,0,0,1
